In [4]:
%%writefile mapper-tf.py
from __future__ import print_function
import sys
import re


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

path = 'stop_words_en.txt'

stopWords = set()

with open(path) as stopWordsFile:
    for line in stopWordsFile:
        try:         
            stopWords.add(unicode(line.strip()))
        except ValueError as e:
            continue

wordSum, stopWordSum = 0, 0

import string

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue

    words = re.split("\W*\s+\W*", text.lower(), flags=re.UNICODE)
    
    wordsInArticle = 0
    
    for word in words:
        if word.strip(string.punctuation) in stopWords:
            stopWordSum += 1
            print("reporter:counter:Wiki stats,Stop words found,%d" % 1, file=sys.stderr)
            continue
        wordSum += 1
        wordsInArticle += 1
        #print >> sys.stderr, "reporter:counter:Wiki stats,Total words ound,%d" % 1
        print("reporter:counter:Wiki stats,Total words found,%d" % 1, file=sys.stderr)
        print(article_id, word.strip(string.punctuation), 1, sep='\t')
    print(article_id, "!wordsInArticle", wordsInArticle, sep='\t')



Overwriting mapper-tf.py


In [5]:
%%writefile reducer-tf.py

import sys

current_article_id = None
current_word = None
word_sum, wordsInArticle = 0, 0

for line in sys.stdin:
    try:
        article_id, word, count = line.strip().split('\t')
        #print "after parse:", article_id, word, count
        count = int(count)
    except ValueError as e:
        #print "Parse error:", e
        continue

    if current_article_id != article_id:
        if current_word != None:
            print "%s\t%s\t%f" % (current_article_id, current_word, 1.0*word_sum/wordsInArticle)
            word_sum = 0
            current_word = word
        wordsInArticle = count  # first article string is always words in article count
        current_article_id = article_id
        #print "wordsInArticle:", word, wordsInArticle
        continue
        
    if current_word != word :
        if current_word != None and current_word != '!wordsInArticle': 
            print "%s\t%s\t%f" % (current_article_id, current_word, 1.0*word_sum/wordsInArticle)
            #print "%s\t%s\t%d\t%d" % (current_article_id, current_word, word_sum, wordsInArticle)
        word_sum = 0
        current_word = word
    word_sum += count

if current_article_id:
    print "%s\t%s\t%f" % (current_article_id, word, 1.0*word_sum/wordsInArticle)


Writing reducer-tf.py


In [1]:
!hdfs dfs -mkdir /data/wiki/test

In [2]:
!hdfs dfs -put wiki-test.txt /data/wiki/test

In [3]:
!hdfs dfs -ls /data/wiki/test

Found 1 items
-rw-r--r--   1 jovyan supergroup        479 2019-02-04 07:35 /data/wiki/test/wiki-test.txt


In [6]:
%%bash

#OUT_DIR="wordcount_result_"$(date +"%s%6N")
OUT_DIR="/data/wiki/test/tf-out"
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.job.output.key.partitioner.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedPartitioner \
    -D mapred.jab.name="Tf-Idf tf task" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper-tf.py,reducer-tf.py,/datasets/stop_words_en.txt  \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.partition.keypartitioner.options="-k1,1" \
    -D mapreduce.partition.keycomparator.options="-k1,2" \
    -mapper "python mapper-tf.py" \
    -reducer "python reducer-tf.py" \
    -input /data/wiki/test \
    -output ${OUT_DIR} > /dev/null


hdfs dfs -cat ${OUT_DIR}/* | head -100

10	3	0.153846
10	behavior	0.076923
10	congrayou	0.076923
10	emulate	0.076923
10	hadoop	0.153846
10	hadoop's	0.076923
10	job	0.153846
10	know	0.076923
10	stages	0.153846
17	0	0.062500
17	article_ids	0.062500
17	bash	0.062500
17	behavior	0.062500
17	code	0.062500
17	emulate	0.062500
17	greater	0.062500
17	hadoop's	0.062500
17	hint	0.125000
17	mapper's	0.062500
17	reducer's	0.062500
17	simple	0.062500
17	using	0.062500
17	wikipedia	0.125000
23	applications	0.125000
23	bash	0.062500
23	congrayou	0.125000
23	debugging	0.125000
23	easily	0.062500
23	emulating	0.062500
23	hadoop’s	0.062500
23	locally	0.062500
23	lot	0.062500
23	safe	0.062500
23	time	0.062500
23	using	0.062500
23	workflow	0.062500


rm: `/data/wiki/test/tf-out': No such file or directory
19/02/04 07:35:59 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/04 07:35:59 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/04 07:36:00 INFO mapred.FileInputFormat: Total input files to process : 1
19/02/04 07:36:01 INFO mapreduce.JobSubmitter: number of splits:2
19/02/04 07:36:01 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1549005095618_0001
19/02/04 07:36:02 INFO impl.YarnClientImpl: Submitted application application_1549005095618_0001
19/02/04 07:36:02 INFO mapreduce.Job: The url to track the job: http://bce6a116ae74:8088/proxy/application_1549005095618_0001/
19/02/04 07:36:02 INFO mapreduce.Job: Running job: job_1549005095618_0001
19/02/04 07:36:10 INFO mapreduce.Job: Job job_1549005095618_0001 running in uber mode : false
19/02/04 07:36:10 INFO mapreduce.Job:  map 0% reduce 0%
19/02/04 07:36:17 INFO mapreduce.Job:  map 100% reduce 0%
19/02/04 07:36:22 INFO 

In [7]:
! hdfs dfs -ls /data/wiki/test/idf-out

ls: `/data/wiki/test/idf-out': No such file or directory


In [8]:
%%writefile mapper-idf.py
from __future__ import print_function
import sys

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

for line in sys.stdin:
    _, word, _ = unicode(line.strip()).split('\t')
    print("reporter:counter:Idf stats,Total words found,%d" % 1, file=sys.stderr)
    print(word, 1, sep='\t')


Writing mapper-idf.py


In [23]:
%%writefile reducer-idf.py
import sys
from math import log

current_key = None
word_sum = 0
for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%f" % (current_key, (1/log(1 + word_sum)))
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%f" % (current_key, (1/log(1 + word_sum)))


Overwriting reducer-idf.py


In [10]:
%%bash

#OUT_DIR="wordcount_result_"$(date +"%s%6N")
OUT_DIR="/data/wiki/test/idf-out"
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Tf-Idf idf task" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper-idf.py,reducer-idf.py \
    -mapper "python mapper-idf.py" \
    -reducer "python reducer-idf.py" \
    -input /data/wiki/test/tf-out \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/* 

0	1
3	1
applications	1
article_ids	1
bash	2
behavior	2
code	1
congrayou	2
debugging	1
easily	1
emulate	2
emulating	1
greater	1
hadoop	1
hadoop's	2
hadoop’s	1
hint	1
job	1
know	1
locally	1
lot	1
mapper's	1
reducer's	1
safe	1
simple	1
stages	1
time	1
using	2
wikipedia	1
workflow	1


rm: `/data/wiki/test/idf-out': No such file or directory
19/02/04 07:38:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/04 07:38:58 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/04 07:38:58 INFO mapred.FileInputFormat: Total input files to process : 1
19/02/04 07:38:59 INFO mapreduce.JobSubmitter: number of splits:2
19/02/04 07:38:59 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1549005095618_0002
19/02/04 07:38:59 INFO impl.YarnClientImpl: Submitted application application_1549005095618_0002
19/02/04 07:38:59 INFO mapreduce.Job: The url to track the job: http://bce6a116ae74:8088/proxy/application_1549005095618_0002/
19/02/04 07:38:59 INFO mapreduce.Job: Running job: job_1549005095618_0002
19/02/04 07:39:05 INFO mapreduce.Job: Job job_1549005095618_0002 running in uber mode : false
19/02/04 07:39:05 INFO mapreduce.Job:  map 0% reduce 0%
19/02/04 07:39:10 INFO mapreduce.Job:  map 100% reduce 0%
19/02/04 07:39:15 INFO

In [11]:
!hdfs dfs -ls /data/wiki/test/idf-out

Found 2 items
-rw-r--r--   1 jovyan supergroup          0 2019-02-04 07:39 /data/wiki/test/idf-out/_SUCCESS
-rw-r--r--   1 jovyan supergroup        282 2019-02-04 07:39 /data/wiki/test/idf-out/part-00000


In [12]:
!hdfs dfs -cat /data/wiki/test/idf-out/*

0	1
3	1
applications	1
article_ids	1
bash	2
behavior	2
code	1
congrayou	2
debugging	1
easily	1
emulate	2
emulating	1
greater	1
hadoop	1
hadoop's	2
hadoop’s	1
hint	1
job	1
know	1
locally	1
lot	1
mapper's	1
reducer's	1
safe	1
simple	1
stages	1
time	1
using	2
wikipedia	1
workflow	1


In [13]:
!hdfs dfs -getmerge  /data/wiki/test/idf-out/* idf.csv

In [14]:
! ls -l

total 160
-rw-r--r-- 1 jovyan users   282 Feb  4 07:57 idf.csv
-rw-r--r-- 1 jovyan users   313 Feb  4 07:38 mapper-idf.py
-rw-r--r-- 1 jovyan users  1239 Feb  4 07:35 mapper-tf.py
-rw-r--r-- 1 jovyan root    868 May  2  2018 README.md
-rw-r--r-- 1 jovyan users   431 Feb  4 07:38 reducer-idf.py
-rw-r--r-- 1 jovyan users  1254 Feb  4 07:35 reducer-tf.py
-rw-r--r-- 1 jovyan users 42127 Feb  1 07:12 StopWordsTask22.ipynb
-rw-r--r-- 1 jovyan root  10072 Aug  1  2018 StopWordsTask2.ipynb
-rw-r--r-- 1 root   users   758 Feb  1 07:11 supervisord.log
-rw-r--r-- 1 root   users     2 Feb  1 07:11 supervisord.pid
-rw-r--r-- 1 jovyan users 20628 Feb  4 07:40 tf-idf-assignment.ipynb
-rw-r--r-- 1 jovyan users   479 Feb  4 07:31 wiki-test.txt
-rw-r--r-- 1 jovyan root   9198 Feb  1 07:13 WordCountTask0.ipynb
-rw-r--r-- 1 jovyan users 20838 Feb  1 08:54 WordsRatingTask11.ipynb
-rw-r--r-- 1 jovyan root   5287 Feb  1 07:51 WordsRatingTask1.ipynb


In [15]:
! cat idf.csv

0	1
3	1
applications	1
article_ids	1
bash	2
behavior	2
code	1
congrayou	2
debugging	1
easily	1
emulate	2
emulating	1
greater	1
hadoop	1
hadoop's	2
hadoop’s	1
hint	1
job	1
know	1
locally	1
lot	1
mapper's	1
reducer's	1
safe	1
simple	1
stages	1
time	1
using	2
wikipedia	1
workflow	1


In [21]:
%%writefile mapper-tf-idf.py
from __future__ import print_function
import sys
import re


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

path = 'idf.csv'

idfWords = []

with open(path) as idfWordsFile:
    for line in idfWordsFile:
        try:
            word, v = line.split('\t')
            idf = int(v)
            idfWords.append([unicode(word.strip()), idf])
        except ValueError as e:
            continue
#print(idfWords)

for line in sys.stdin:
    try:
        article_id, tfWord, v = unicode(line.strip()).split('\t')
        tf = float(v)
    except ValueError as e:
        continue

    idfWord = next((w for w in idfWords if tfWord in w), [])

    if len(idfWord) == 0:
        print("reporter:counter:Wiki stats,Unfound tf into idf words,%d" % 1, file=sys.stderr)
        countinue
    idf = idfWord[1]
    print("reporter:counter:Wiki stats,tf*idf words,%d" % 1, file=sys.stderr)
    #print(tfWord, article_id, tf, idf, tf*idf, sep='\t')
    print(tfWord, article_id, tf*idf, sep='\t')


Overwriting mapper-tf-idf.py


In [22]:
%%bash

#OUT_DIR="wordcount_result_"$(date +"%s%6N")
OUT_DIR="/data/wiki/test/tf-idf-out"
NUM_REDUCERS=0

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Tf-Idf tf*idf task" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper-tf-idf.py,idf.csv \
    -mapper "python mapper-tf-idf.py" \
    -input /data/wiki/test/tf-out \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/* 

reducer's	17	0.0625
simple	17	0.0625
using	17	0.125
wikipedia	17	0.125
applications	23	0.125
bash	23	0.125
congrayou	23	0.25
debugging	23	0.125
easily	23	0.0625
emulating	23	0.0625
hadoop’s	23	0.0625
locally	23	0.0625
lot	23	0.0625
safe	23	0.0625
time	23	0.0625
using	23	0.125
workflow	23	0.0625
3	10	0.153846
behavior	10	0.153846
congrayou	10	0.153846
emulate	10	0.153846
hadoop	10	0.153846
hadoop's	10	0.153846
job	10	0.153846
know	10	0.076923
stages	10	0.153846
0	17	0.0625
article_ids	17	0.0625
bash	17	0.125
behavior	17	0.125
code	17	0.0625
emulate	17	0.125
greater	17	0.0625
hadoop's	17	0.125
hint	17	0.125
mapper's	17	0.0625


19/02/04 09:59:38 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/04 09:59:38 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/04 09:59:39 INFO mapred.FileInputFormat: Total input files to process : 1
19/02/04 09:59:40 INFO mapreduce.JobSubmitter: number of splits:2
19/02/04 09:59:40 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1549005095618_0006
19/02/04 09:59:40 INFO impl.YarnClientImpl: Submitted application application_1549005095618_0006
19/02/04 09:59:40 INFO mapreduce.Job: The url to track the job: http://bce6a116ae74:8088/proxy/application_1549005095618_0006/
19/02/04 09:59:40 INFO mapreduce.Job: Running job: job_1549005095618_0006
19/02/04 09:59:46 INFO mapreduce.Job: Job job_1549005095618_0006 running in uber mode : false
19/02/04 09:59:46 INFO mapreduce.Job:  map 0% reduce 0%
19/02/04 09:59:51 INFO mapreduce.Job:  map 50% reduce 0%
19/02/04 09:59:52 INFO mapreduce.Job:  map 100% reduce 0%
19/02/04 09:59:53 INFO